<a href="https://colab.research.google.com/github/jm-menon/QKD-Blockchain-Security-System/blob/Combined-Code/QKD%2BAES%2BBLOCKCHAIN_CYCLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === QKD-AES ENCRYPTION + BLOCKCHAIN LINKED CHAIN DEMO ===
import os
import random
import hashlib
import time
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding


# ------------------------------------------------------------
# PART 1: QKD Key Simulation + AES Encryption
# ------------------------------------------------------------

def simulate_qkd(key_size: int):
    alice_key = bytearray(os.urandom(key_size // 8))
    bob_key = bytearray(os.urandom(key_size // 8))

    # Introduce random bit errors (5%)
    error_rate = 0.05
    for i in random.sample(range(len(alice_key)), int(error_rate * len(alice_key))):
        alice_key[i] ^= 0x01  # Flip 1 bit

    # Correct errors (simple demo)
    corrected_key = correct_errors(alice_key, bob_key)
    return bytes(corrected_key), bytes(bob_key)


def correct_errors(alice_key, bob_key):
    corrected = bytearray(alice_key)
    for i in range(len(corrected)):
        if corrected[i] != bob_key[i]:
            corrected[i] = bob_key[i]
    return corrected


def encrypt(message: str, key: bytes) -> str:
    backend = default_backend()
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(message.encode()) + padder.finalize()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return base64.b64encode(ciphertext).decode()


def decrypt(ciphertext_b64: str, key: bytes) -> str:
    backend = default_backend()
    ciphertext = base64.b64decode(ciphertext_b64.encode())
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()
    return data.decode()


# ------------------------------------------------------------
# PART 2: Blockchain + IPFS Simulation
# ------------------------------------------------------------

class Block:
    """Represents a single block in the blockchain."""
    def __init__(self, data, previous_hash):
        self.timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        sha = hashlib.sha256()
        sha.update(self.timestamp.encode())
        sha.update(self.data.encode())
        sha.update(self.previous_hash.encode())
        return sha.hexdigest()

    def __str__(self):
        return (f"Block:\n"
                f"  Timestamp       : {self.timestamp}\n"
                f"  Data (IPFS hash): {self.data}\n"
                f"  Block Hash      : {self.hash}\n"
                f"  Prev Block Hash : {self.previous_hash}\n")


class Blockchain:
    """Simple blockchain with integrity validation."""
    def __init__(self):
        self.chain = []
        self.genesis_block()

    def genesis_block(self):
        genesis = Block("Genesis Block", "0")
        self.chain.append(genesis)

    def add_block(self, data):
        prev_block = self.chain[-1]
        new_block = Block(data, prev_block.hash)
        self.chain.append(new_block)

    def is_valid(self):
        for i in range(1, len(self.chain)):
            curr = self.chain[i]
            prev = self.chain[i - 1]
            if curr.hash != curr.calculate_hash() or curr.previous_hash != prev.hash:
                return False
        return True

    def display_chain(self):
        """Pretty-print the full blockchain with link arrows."""
        print("\n=== BLOCKCHAIN STRUCTURE ===")
        for i, block in enumerate(self.chain):
            print(block)
            if i < len(self.chain) - 1:
                print("        ↓")
                print("       LINKED TO PREVIOUS BLOCK")
                print("        ↓")
        print("============================\n")


class IPFS:
    """Simulated IPFS for storing data by hash."""
    def __init__(self):
        self.storage = {}

    def store(self, data):
        data_hash = hashlib.sha256(data.encode()).hexdigest()
        self.storage[data_hash] = data
        return data_hash


# ------------------------------------------------------------
# PART 3: Integrated AES + Blockchain Demonstration
# ------------------------------------------------------------

def main():
    print("\n=== QKD-AES ENCRYPTION + LINKED BLOCKCHAIN DEMO ===")
    key_size = 128
    encryption_key, _ = simulate_qkd(key_size)

    blockchain = Blockchain()
    ipfs = IPFS()

    while True:
        user_input = input("\nEnter a message to encrypt (or 'q' to quit): ")
        if user_input.lower() == 'q':
            break

        # Encrypt/decrypt message locally
        ciphertext_b64 = encrypt(user_input, encryption_key)
        decrypted = decrypt(ciphertext_b64, encryption_key)

        print(f"\nEncrypted (Base64): {ciphertext_b64}")
        print(f"Decrypted         : {decrypted}")

        # Store encrypted data in IPFS + Blockchain
        data_hash = ipfs.store(ciphertext_b64)
        blockchain.add_block(data_hash)

        # Blockchain Output
        print("\n[BLOCKCHAIN EVENT]")
        print(f"Data stored in IPFS hash : {data_hash}")
        print(f"New block hash           : {blockchain.chain[-1].hash}")
        print(f"Previous block hash      : {blockchain.chain[-1].previous_hash}")
        print(f"Blockchain valid?        : {blockchain.is_valid()}")

        # Display chain structure visually
        blockchain.display_chain()

    print("\n=== FINAL BLOCKCHAIN STATE ===")
    blockchain.display_chain()


if __name__ == "__main__":
    main()
